- https://medium.com/stanford-cs224w/introducing-distmult-and-complex-for-pytorch-geometric-6f40974223d0
Knowledge graph embeddings are a powerful tool for analyzing knowledge graphs, and now with our implementation of DistMult and ComplEx, they can be easily integrated into any project! In this post, we’ll walk you through the intuition behind DistMult and ComplEx, and show you how to use PyG to implement these techniques, and explore some practical applications of knowledge graph embeddings in graph machine learning. So, get ready to take your graph analysis skills to the next level with DistMult and ComplEx in PyG!

- https://github.com/pyg-team/pytorch_geometric/blob/master/examples/kge_fb15k_237.py

In [2]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=0eadf4239804384634b72eb49fce46c41b470fba4f5ce14bd95602ef60ce7764
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [4]:
!pip install --upgrade torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import torch as Tensor

In [18]:
# import argparse
# import os.path as osp

# import torch
# import torch.optim as optim

# from torch_geometric.datasets import FB15k_237
# from torch_geometric.nn import ComplEx, DistMult, TransE

# model_map = {
#     'transe': TransE,
#     'complex': ComplEx,
#     'distmult': DistMult,
# }

In [ ]:
%tb

# parser = argparse.ArgumentParser()
# parser.add_argument('-f')
# parser.add_argument('--model', choices=model_map.keys(), type=str.lower,
#                     required=True)
# args = parser.parse_args()

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'FB15k')

# train_data = FB15k_237(path, split='train')[0].to(device)
# val_data = FB15k_237(path, split='val')[0].to(device)
# test_data = FB15k_237(path, split='test')[0].to(device)

# model_arg_map = {'rotate': {'margin': 9.0}}
# model = model_map[args.model](
#     num_nodes=train_data.num_nodes,
#     num_relations=train_data.num_edge_types,
#     hidden_channels=50,
#     **model_arg_map.get(args.model, {}),
# ).to(device)

# loader = model.loader(
#     head_index=train_data.edge_index[0],
#     rel_type=train_data.edge_type,
#     tail_index=train_data.edge_index[1],
#     batch_size=1000,
#     shuffle=True,
# )

# optimizer_map = {
#     'transe': optim.Adam(model.parameters(), lr=0.01),
#     'complex': optim.Adagrad(model.parameters(), lr=0.001, weight_decay=1e-6),
#     'distmult': optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-6),

# }
# optimizer = optimizer_map[args.model]


# def train():
#     model.train()
#     total_loss = total_examples = 0
#     for head_index, rel_type, tail_index in loader:
#         optimizer.zero_grad()
#         loss = model.loss(head_index, rel_type, tail_index)
#         loss.backward()
#         optimizer.step()
#         total_loss += float(loss) * head_index.numel()
#         total_examples += head_index.numel()
#     return total_loss / total_examples


# @torch.no_grad()
# def test(data):
#     model.eval()
#     return model.test(
#         head_index=data.edge_index[0],
#         rel_type=data.edge_type,
#         tail_index=data.edge_index[1],
#         batch_size=20000,
#         k=10,
#     )


# for epoch in range(1, 501):
#     loss = train()
#     print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
#     if epoch % 25 == 0:
#         rank, hits = test(val_data)
#         print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
#               f'Val Hits@10: {hits:.4f}')

# rank, hits_at_10 = test(test_data)
# print(f'Test Mean Rank: {rank:.2f}, Test Hits@10: {hits_at_10:.4f}')

In [38]:
class KGEModel(torch.nn.Module):

  def __init__(
          self,
          num_nodes: int,
          num_relations: int,
          hidden_channels: int,
          sparse: bool = False,
      ):
          super().__init__()

          self.num_nodes = num_nodes
          self.num_relations = num_relations
          self.hidden_channels = hidden_channels

          self.node_emb = Embedding(num_nodes, hidden_channels, sparse=sparse)
          self.rel_emb = Embedding(num_relations, hidden_channels, sparse=sparse)

In [36]:
# DistMult forward

def forward(
    self,
    head_index: Tensor,
    rel_type: Tensor,
    tail_index: Tensor,
) -> Tensor:

    head = self.node_emb(head_index)
    rel = self.rel_emb(rel_type)
    tail = self.node_emb(tail_index)

    return (head * rel * tail).sum(dim=-1)

In [35]:
# DistMult loss

def loss(
    self,
    head_index: Tensor,
    rel_type: Tensor,
    tail_index: Tensor,
) -> Tensor:

    pos_score = self(head_index, rel_type, tail_index)
    neg_score = self(*self.random_sample(head_index, rel_type, tail_index))

    return F.margin_ranking_loss(
        pos_score,
        neg_score,
        target=torch.ones_like(pos_score),
        margin=self.margin,
    )

In [34]:
class ComplEx(KGEModel):

  def __init__(
      self,
      num_nodes: int,
      num_relations: int,
      hidden_channels: int,
      sparse: bool = False,
  ):
      super().__init__(num_nodes, num_relations, hidden_channels, sparse)

      self.node_emb_im = Embedding(num_nodes, hidden_channels, sparse=sparse)
      self.rel_emb_im = Embedding(num_relations, hidden_channels,
                                  sparse=sparse)

      self.reset_parameters()

In [33]:
# ComplEx forward

def forward(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ) -> Tensor:

        head_re = self.node_emb(head_index)
        head_im = self.node_emb_im(head_index)
        rel_re = self.rel_emb(rel_type)
        rel_im = self.rel_emb_im(rel_type)
        tail_re = self.node_emb(tail_index)
        tail_im = self.node_emb_im(tail_index)

        return (triple_dot(head_re, rel_re, tail_re) +
                triple_dot(head_im, rel_re, tail_im) +
                triple_dot(head_re, rel_im, tail_im) -
                triple_dot(head_im, rel_im, tail_re))

def triple_dot(x: Tensor, y: Tensor, z: Tensor) -> Tensor:
    return (x * y * z).sum(dim=-1)

In [32]:
# ComplEx loss

def loss(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ) -> Tensor:

        pos_score = self(head_index, rel_type, tail_index)
        neg_score = self(*self.random_sample(head_index, rel_type, tail_index))
        scores = torch.cat([pos_score, neg_score], dim=0)

        pos_target = torch.ones_like(pos_score)
        neg_target = torch.zeros_like(neg_score)
        target = torch.cat([pos_target, neg_target], dim=0)

        return F.binary_cross_entropy_with_logits(scores, target)

In [48]:
from torch_geometric.datasets import FB15k_237
import os.path as osp
from google.colab import drive

drive.mount('/content/drive')
path = osp.join('/content/drive/MyDrive', 'data', 'FB15k')

Mounted at /content/drive


In [49]:
train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

Processing...
Done!


In [55]:
import os.path as osp

import torch
import torch.optim as optim

from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import ComplEx

device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = osp.join(osp.dirname(r'/content/drive/MyDrive/'), 'data', 'FB15k')

train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

In [56]:
model = ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations=train_data.num_edge_types,
    hidden_channels=50,
).to(device)

In [57]:
loader = model.loader(
    head_index=train_data.edge_index[0],
    rel_type=train_data.edge_type,
    tail_index=train_data.edge_index[1],
    batch_size=1000,
    shuffle=True,
)

In [58]:
optimizer = optim.Adagrad(model.parameters(), lr=0.001, weight_decay=1e-6)

In [59]:
def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples

In [25]:
@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )

In [ ]:
for epoch in range(1, 501):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 25 == 0:
        rank, hits = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val Hits@10: {hits:.4f}')

rank, hits_at_10 = test(test_data)
print(f'Test Mean Rank: {rank:.2f}, Test Hits@10: {hits_at_10:.4f}')

Epoch: 001, Loss: 0.6931
Epoch: 002, Loss: 0.6931
Epoch: 003, Loss: 0.6931
Epoch: 004, Loss: 0.6931
Epoch: 005, Loss: 0.6931
Epoch: 006, Loss: 0.6931
Epoch: 007, Loss: 0.6931
Epoch: 008, Loss: 0.6930
Epoch: 009, Loss: 0.6929
Epoch: 010, Loss: 0.6928
Epoch: 011, Loss: 0.6927
Epoch: 012, Loss: 0.6925
Epoch: 013, Loss: 0.6923
Epoch: 014, Loss: 0.6920
Epoch: 015, Loss: 0.6916
Epoch: 016, Loss: 0.6912
Epoch: 017, Loss: 0.6907
Epoch: 018, Loss: 0.6902
Epoch: 019, Loss: 0.6896
Epoch: 020, Loss: 0.6889
Epoch: 021, Loss: 0.6882
Epoch: 022, Loss: 0.6874
Epoch: 023, Loss: 0.6865
Epoch: 024, Loss: 0.6856
Epoch: 025, Loss: 0.6847


100%|██████████| 17535/17535 [04:26<00:00, 65.81it/s]


Epoch: 025, Val Mean Rank: 717.91, Val Hits@10: 0.3046
Epoch: 026, Loss: 0.6837
Epoch: 027, Loss: 0.6826
Epoch: 028, Loss: 0.6816
Epoch: 029, Loss: 0.6804
Epoch: 030, Loss: 0.6792
Epoch: 031, Loss: 0.6780
Epoch: 032, Loss: 0.6768
Epoch: 033, Loss: 0.6755
Epoch: 034, Loss: 0.6742
Epoch: 035, Loss: 0.6728
Epoch: 036, Loss: 0.6715
Epoch: 037, Loss: 0.6701
Epoch: 038, Loss: 0.6687
Epoch: 039, Loss: 0.6672
Epoch: 040, Loss: 0.6658
Epoch: 041, Loss: 0.6643
Epoch: 042, Loss: 0.6628
Epoch: 043, Loss: 0.6613
Epoch: 044, Loss: 0.6597
Epoch: 045, Loss: 0.6582
Epoch: 046, Loss: 0.6568
Epoch: 047, Loss: 0.6552
Epoch: 048, Loss: 0.6537
Epoch: 049, Loss: 0.6521
Epoch: 050, Loss: 0.6505


100%|██████████| 17535/17535 [04:10<00:00, 70.01it/s]


Epoch: 050, Val Mean Rank: 716.12, Val Hits@10: 0.3144
Epoch: 051, Loss: 0.6490
Epoch: 052, Loss: 0.6474
Epoch: 053, Loss: 0.6459
Epoch: 054, Loss: 0.6442
Epoch: 055, Loss: 0.6427
Epoch: 056, Loss: 0.6412
Epoch: 057, Loss: 0.6396
Epoch: 058, Loss: 0.6381
Epoch: 059, Loss: 0.6366
Epoch: 060, Loss: 0.6351
Epoch: 061, Loss: 0.6335
Epoch: 062, Loss: 0.6320
Epoch: 063, Loss: 0.6305
Epoch: 064, Loss: 0.6290
Epoch: 065, Loss: 0.6275
Epoch: 066, Loss: 0.6260
Epoch: 067, Loss: 0.6246
Epoch: 068, Loss: 0.6231
Epoch: 069, Loss: 0.6217
Epoch: 070, Loss: 0.6203
Epoch: 071, Loss: 0.6188
Epoch: 072, Loss: 0.6175
Epoch: 073, Loss: 0.6162
Epoch: 074, Loss: 0.6145
Epoch: 075, Loss: 0.6132


...

Epoch: 475, Val Mean Rank: 550.35, Val Hits@10: 0.3537
Epoch: 476, Loss: 0.4713
Epoch: 477, Loss: 0.4712
Epoch: 478, Loss: 0.4711
Epoch: 479, Loss: 0.4713
Epoch: 480, Loss: 0.4702
Epoch: 481, Loss: 0.4703
Epoch: 482, Loss: 0.4703
Epoch: 483, Loss: 0.4707
Epoch: 484, Loss: 0.4698
Epoch: 485, Loss: 0.4707
Epoch: 486, Loss: 0.4708
Epoch: 487, Loss: 0.4696
Epoch: 488, Loss: 0.4699
Epoch: 489, Loss: 0.4702
Epoch: 490, Loss: 0.4703
Epoch: 491, Loss: 0.4700
Epoch: 492, Loss: 0.4689
Epoch: 493, Loss: 0.4694
Epoch: 494, Loss: 0.4697
Epoch: 495, Loss: 0.4689
Epoch: 496, Loss: 0.4692
Epoch: 497, Loss: 0.4682
Epoch: 498, Loss: 0.4698
Epoch: 499, Loss: 0.4691
Epoch: 500, Loss: 0.4684


100%|██████████| 17535/17535 [04:22<00:00, 66.71it/s]


Epoch: 500, Val Mean Rank: 545.41, Val Hits@10: 0.3547


100%|██████████| 20466/20466 [05:03<00:00, 67.34it/s]

Test Mean Rank: 552.18, Test Hits@10: 0.3502
